In [18]:
import pandas as pd
from tqdm import tqdm
import os
from saveAndLoad import *

def printAllHead(df,n=5):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
        print(df.head(n))
def printAll(x):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
        print(x)

def parse_info(info_str):
    """Parse the INFO string from the VCF into a dictionary."""
    info_dict = {}
    for field in info_str.split(';'):
        if '=' in field:
            key, val = field.split('=', 1)
            info_dict[key] = val
        else:
            # In cases where there's a key with no '=' (some VCFs have flags)
            info_dict[field] = True
    return pd.Series(info_dict)

def inspect(df,c):
    print(df.columns)
    print('nunique',df[c].nunique())
    print(df[c].unique())
    print(df[c].value_counts())


In [3]:
###variationPropertyDocumentationUrl=ftp://ftp.ncbi.nlm.nih.gov/snp/specs/dbSNP_BitField_latest.pdf	
vcf_info = '''
##INFO=<ID=RS,Number=1,Type=Integer,Description="dbSNP ID (i.e. rs number)">
##INFO=<ID=RSPOS,Number=1,Type=Integer,Description="Chr position reported in dbSNP">
##INFO=<ID=RV,Number=0,Type=Flag,Description="RS orientation is reversed">
##INFO=<ID=VP,Number=1,Type=String,Description="Variation Property.  Documentation is at ftp://ftp.ncbi.nlm.nih.gov/snp/specs/dbSNP_BitField_latest.pdf">
##INFO=<ID=GENEINFO,Number=1,Type=String,Description="Pairs each of gene symbol:gene id.  The gene symbol and id are delimited by a colon (:) and each pair is delimited by a vertical bar (|)">
##INFO=<ID=dbSNPBuildID,Number=1,Type=Integer,Description="First dbSNP Build for RS">
##INFO=<ID=SAO,Number=1,Type=Integer,Description="Variant Allele Origin: 0 - unspecified, 1 - Germline, 2 - Somatic, 3 - Both">
##INFO=<ID=SSR,Number=1,Type=Integer,Description="Variant Suspect Reason Codes (may be more than one value added together) 0 - unspecified, 1 - Paralog, 2 - byEST, 4 - oldAlign, 8 - Para_EST, 16 - 1kg_failed, 1024 - other">
##INFO=<ID=WGT,Number=1,Type=Integer,Description="Weight, 00 - unmapped, 1 - weight 1, 2 - weight 2, 3 - weight 3 or more">
##INFO=<ID=VC,Number=1,Type=String,Description="Variation Class">
##INFO=<ID=PM,Number=0,Type=Flag,Description="Variant is Precious(Clinical,Pubmed Cited)">
##INFO=<ID=TPA,Number=0,Type=Flag,Description="Provisional Third Party Annotation(TPA) (currently rs from PHARMGKB who will give phenotype data)">
##INFO=<ID=PMC,Number=0,Type=Flag,Description="Links exist to PubMed Central article">
##INFO=<ID=S3D,Number=0,Type=Flag,Description="Has 3D structure - SNP3D table">
##INFO=<ID=SLO,Number=0,Type=Flag,Description="Has SubmitterLinkOut - From SNP->SubSNP->Batch.link_out">
##INFO=<ID=NSF,Number=0,Type=Flag,Description="Has non-synonymous frameshift A coding region variation where one allele in the set changes all downstream amino acids. FxnClass = 44">
##INFO=<ID=NSM,Number=0,Type=Flag,Description="Has non-synonymous missense A coding region variation where one allele in the set changes protein peptide. FxnClass = 42">
##INFO=<ID=NSN,Number=0,Type=Flag,Description="Has non-synonymous nonsense A coding region variation where one allele in the set changes to STOP codon (TER). FxnClass = 41">
##INFO=<ID=REF,Number=0,Type=Flag,Description="Has reference A coding region variation where one allele in the set is identical to the reference sequence. FxnCode = 8">
##INFO=<ID=SYN,Number=0,Type=Flag,Description="Has synonymous A coding region variation where one allele in the set does not change the encoded amino acid. FxnCode = 3">
##INFO=<ID=U3,Number=0,Type=Flag,Description="In 3' UTR Location is in an untranslated region (UTR). FxnCode = 53">
##INFO=<ID=U5,Number=0,Type=Flag,Description="In 5' UTR Location is in an untranslated region (UTR). FxnCode = 55">
##INFO=<ID=ASS,Number=0,Type=Flag,Description="In acceptor splice site FxnCode = 73">
##INFO=<ID=DSS,Number=0,Type=Flag,Description="In donor splice-site FxnCode = 75">
##INFO=<ID=INT,Number=0,Type=Flag,Description="In Intron FxnCode = 6">
##INFO=<ID=R3,Number=0,Type=Flag,Description="In 3' gene region FxnCode = 13">
##INFO=<ID=R5,Number=0,Type=Flag,Description="In 5' gene region FxnCode = 15">
##INFO=<ID=OTH,Number=0,Type=Flag,Description="Has other variant with exactly the same set of mapped positions on NCBI refernce assembly.">
##INFO=<ID=CFL,Number=0,Type=Flag,Description="Has Assembly conflict. This is for weight 1 and 2 variant that maps to different chromosomes on different assemblies.">
##INFO=<ID=ASP,Number=0,Type=Flag,Description="Is Assembly specific. This is set if the variant only maps to one assembly">
##INFO=<ID=MUT,Number=0,Type=Flag,Description="Is mutation (journal citation, explicit fact): a low frequency variation that is cited in journal and other reputable sources">
##INFO=<ID=VLD,Number=0,Type=Flag,Description="Is Validated.  This bit is set if the variant has 2+ minor allele count based on frequency or genotype data.">
##INFO=<ID=G5A,Number=0,Type=Flag,Description=">5% minor allele frequency in each and all populations">
##INFO=<ID=G5,Number=0,Type=Flag,Description=">5% minor allele frequency in 1+ populations">
##INFO=<ID=HD,Number=0,Type=Flag,Description="Marker is on high density genotyping kit (50K density or greater).  The variant may have phenotype associations present in dbGaP.">
##INFO=<ID=GNO,Number=0,Type=Flag,Description="Genotypes available. The variant has individual genotype (in SubInd table).">
##INFO=<ID=KGPhase1,Number=0,Type=Flag,Description="1000 Genome phase 1 (incl. June Interim phase 1)">
##INFO=<ID=KGPhase3,Number=0,Type=Flag,Description="1000 Genome phase 3">
##INFO=<ID=CDA,Number=0,Type=Flag,Description="Variation is interrogated in a clinical diagnostic assay">
##INFO=<ID=LSD,Number=0,Type=Flag,Description="Submitted from a locus-specific database">
##INFO=<ID=MTP,Number=0,Type=Flag,Description="Microattribution/third-party annotation(TPA:GWAS,PAGE)">
##INFO=<ID=OM,Number=0,Type=Flag,Description="Has OMIM/OMIA">
##INFO=<ID=NOC,Number=0,Type=Flag,Description="Contig allele not present in variant allele list. The reference sequence allele at the mapped position is not present in the variant allele list, adjusted for orientation.">
##INFO=<ID=WTD,Number=0,Type=Flag,Description="Is Withdrawn by submitter If one member ss is withdrawn by submitter, then this bit is set.  If all member ss' are withdrawn, then the rs is deleted to SNPHistory">
##INFO=<ID=NOV,Number=0,Type=Flag,Description="Rs cluster has non-overlapping allele sets. True when rs set has more than 2 alleles from different submissions and these sets share no alleles in common.">
##FILTER=<ID=NC,Description="Inconsistent Genotype Submission For At Least One Sample">
##INFO=<ID=CAF,Number=.,Type=String,Description="An ordered, comma delimited list of allele frequencies based on 1000Genomes, starting with the reference allele followed by alternate alleles as ordered in the ALT column. Where a 1000Genomes alternate allele is not in the dbSNPs alternate allele set, the allele is added to the ALT column. The minor allele is the second largest value in the list, and was previuosly reported in VCF as the GMAF. This is the GMAF reported on the RefSNP and EntrezSNP pages and VariationReporter">
##INFO=<ID=COMMON,Number=1,Type=Integer,Description="RS is a common SNP.  A common SNP is one that has at least one 1000Genomes population with a minor allele of frequency >= 1% and for which 2 or more founders contribute to that minor allele frequency.">
##INFO=<ID=TOPMED,Number=.,Type=String,Description="An ordered, comma delimited list of allele frequencies based on TOPMed, starting with the reference allele followed by alternate alleles as ordered in the ALT column. The TOPMed minor allele is the second largest value in the list.">
'''

In [4]:
# # https://ftp.ncbi.nih.gov/snp/organisms/human_9606_b150_GRCh38p7/VCF/
# vcf = pd.read_csv('00-common_all.vcf', 
#                 sep='\t', 
#                 comment='#',
#                 chunksize=2e6)
# z = []
# for i in tqdm(vcf):
#     info_df = i['INFO'].apply(parse_info)
#     vcf_info = pd.concat([i.drop(columns='INFO'), info_df], axis=1)
#     print(len(vcf_info))
#     z.append(vcf_info)

# for ni,i in enumerate(z):
#     i.to_csv(f'00-common_all/00-common_all_{int(ni*2e6)}_{int(ni*2e6+len(i))}.csv', index=False)

z = []
for i in os.listdir('00-common_all'):
    z.append(pd.read_csv(f'00-common_all/{i}'))


/tmp/ipykernel_1833114/4215660741.py:18: DtypeWarning: Columns (16,30,31,32,33,35,36,37,38,39,41,42,43,44,45,46,47,48,49,50) have mixed types. Specify dtype option on import or set low_memory=False.
  z.append(pd.read_csv(f'00-common_all/{i}'))
/tmp/ipykernel_1833114/4215660741.py:18: DtypeWarning: Columns (17,18,27,32,34,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50) have mixed types. Specify dtype option on import or set low_memory=False.
  z.append(pd.read_csv(f'00-common_all/{i}'))
/tmp/ipykernel_1833114/4215660741.py:18: DtypeWarning: Columns (13,17,18,27,31,32,33,34,37,38,39,40,41,42,43,44,45,46,47,48,49,50) have mixed types. Specify dtype option on import or set low_memory=False.
  z.append(pd.read_csv(f'00-common_all/{i}'))
/tmp/ipykernel_1833114/4215660741.py:18: DtypeWarning: Columns (0,16,30,31,32,33,34,35,36,37,39,40,41,42,43,45,46,47,48,49,50,51) have mixed types. Specify dtype option on import or set low_memory=False.
  z.append(pd.read_csv(f'00-common_all/{i}'))
/tmp/ipyk

In [5]:
positives = pd.DataFrame()
for i in z:
    nsm = i[~i['NSM'].isna()]
    positives = pd.concat([positives,nsm])

print(len(positives))


181457


In [6]:
positives.columns
inspect(positives,'GENEINFO')

Index(['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'RS', 'RSPOS',
       'dbSNPBuildID', 'SSR', 'SAO', 'VP', 'WGT', 'VC', 'SLO', 'ASP', 'VLD',
       'G5A', 'G5', 'GNO', 'KGPhase1', 'KGPhase3', 'CAF', 'COMMON', 'TOPMED',
       'RV', 'HD', 'NOV', 'GENEINFO', 'R3', 'U3', 'REF.1', 'SYN', 'INT', 'PM',
       'PMC', 'NSM', 'S3D', 'U5', 'R5', 'MTP', 'NOC', 'OTH', 'NSN', 'LSD',
       'OM', 'ASS', 'DSS', 'NSF', 'TPA', 'CFL'],
      dtype='object')
nunique 23259
['DLC1:10395' 'DLC1:10395|C8orf48:157773' 'SGCZ:137868' ...
 'LOC107984347:107984347|PPFIA1:8500' 'PPFIA1:8500|LOC107984347:107984347'
 'CTTN:2017']
GENEINFO
MUC4:4585                                                    529
AHNAK2:113146                                                520
MUC16:94025                                                  434
TTN:7273                                                     289
TTN:7273|TTN-AS1:100506866                                   282
                                               

In [7]:
from Bio import SeqIO
from Bio.Seq import Seq

#records.id, records.name, records.description, records.seq

# wget ftp://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_45/GRCh38.primary_assembly.genome.fa.gz
human_genome_38 = list(SeqIO.parse("../reference_genome/GRCh38.primary_assembly.genome.fa", "fasta"))

## create dictionary of Chr : fasta_seq 
## add 'MT' as synonym for M (to match names in mutations/bed)
hg38_fasta = {i.id.strip('chr'):str(i.seq) for i in human_genome_38}

In [8]:
hg38_fasta['1'][10641:10642]

'G'

In [9]:
cmc = pd.read_csv('CancerMutationCensus_AllData_Tsv_v101_GRCh37/CancerMutationCensus_AllData_v101_GRCh37.tsv', sep='\t')

/tmp/ipykernel_1833114/2230233322.py:1: DtypeWarning: Columns (2,56) have mixed types. Specify dtype option on import or set low_memory=False.
  cmc = pd.read_csv('CancerMutationCensus_AllData_Tsv_v101_GRCh37/CancerMutationCensus_AllData_v101_GRCh37.tsv', sep='\t')


In [10]:
inspect(cmc,'Mutation Description AA')

Index(['GENE_NAME', 'ACCESSION_NUMBER', 'ONC_TSG', 'CGC_TIER', 'MUTATION_URL',
       'LEGACY_MUTATION_ID', 'Mutation CDS', 'Mutation AA', 'AA_MUT_START',
       'AA_MUT_STOP', 'SHARED_AA', 'GENOMIC_WT_ALLELE_SEQ',
       'GENOMIC_MUT_ALLELE_SEQ', 'AA_WT_ALLELE_SEQ', 'AA_MUT_ALLELE_SEQ',
       'Mutation Description CDS', 'Mutation Description AA',
       'ONTOLOGY_MUTATION_CODE', 'GENOMIC_MUTATION_ID',
       'Mutation genome position GRCh37', 'Mutation genome position GRCh38',
       'COSMIC_SAMPLE_TESTED', 'COSMIC_SAMPLE_MUTATED', 'DISEASE',
       'WGS_DISEASE', 'EXAC_AF', 'EXAC_AFR_AF', 'EXAC_AMR_AF', 'EXAC_ADJ_AF',
       'EXAC_EAS_AF', 'EXAC_FIN_AF', 'EXAC_NFE_AF', 'EXAC_SAS_AF',
       'GNOMAD_EXOMES_AF', 'GNOMAD_EXOMES_AFR_AF', 'GNOMAD_EXOMES_AMR_AF',
       'GNOMAD_EXOMES_ASJ_AF', 'GNOMAD_EXOMES_EAS_AF', 'GNOMAD_EXOMES_FIN_AF',
       'GNOMAD_EXOMES_NFE_AF', 'GNOMAD_EXOMES_SAS_AF', 'GNOMAD_GENOMES_AF',
       'GNOMAD_GENOMES_AFR_AF', 'GNOMAD_GENOMES_AMI_AF',
       'GNOMAD_GE

In [23]:
negatives = cmc[~cmc['Mutation Description AA'].isna()]
negatives = negatives[negatives['Mutation Description AA'].str.contains('in frame|inframe|missense', case=False)]
len(negatives)

printAllHead(negatives)

  GENE_NAME   ACCESSION_NUMBER ONC_TSG  CGC_TIER  \
1      FAF1  ENST00000396153.2     NaN       NaN   
4      FAF1  ENST00000396153.2     NaN       NaN   
5      FAF1  ENST00000396153.2     NaN       NaN   
6      FAF1  ENST00000396153.2     NaN       NaN   
7      FAF1  ENST00000396153.2     NaN       NaN   

                                                       MUTATION_URL  \
1  https://cancer.sanger.ac.uk/cosmic/mutation/overview?id=48304295   
4  https://cancer.sanger.ac.uk/cosmic/mutation/overview?id=48301048   
5  https://cancer.sanger.ac.uk/cosmic/mutation/overview?id=48304683   
6  https://cancer.sanger.ac.uk/cosmic/mutation/overview?id=48296921   
7  https://cancer.sanger.ac.uk/cosmic/mutation/overview?id=48296641   

  LEGACY_MUTATION_ID Mutation CDS Mutation AA  AA_MUT_START  AA_MUT_STOP  \
1        COSM7986191     c.711G>T     p.W237C           237          237   
4       COSM10419714     c.797G>A     p.G266E           266          266   
5        COSM7127636    c.1219A>

In [12]:
# There are only 99 genes and 566 missense mutations that are tier 1 or 2 in significance, not enough for contrastive learning
# Conclusion: Use all mutations that are not positives as negatives, validate with correlation between emb distance and phastcon score after contrastive learning
negatives = cmc[cmc['MUTATION_SIGNIFICANCE_TIER'].isin(['1','2'])]
print(len(negatives))
inspect(negatives[negatives['Mutation Description AA'].str.contains('missense',case=False)],'GENE_NAME')
inspect(negatives,'Mutation Description AA')

2431
Index(['GENE_NAME', 'ACCESSION_NUMBER', 'ONC_TSG', 'CGC_TIER', 'MUTATION_URL',
       'LEGACY_MUTATION_ID', 'Mutation CDS', 'Mutation AA', 'AA_MUT_START',
       'AA_MUT_STOP', 'SHARED_AA', 'GENOMIC_WT_ALLELE_SEQ',
       'GENOMIC_MUT_ALLELE_SEQ', 'AA_WT_ALLELE_SEQ', 'AA_MUT_ALLELE_SEQ',
       'Mutation Description CDS', 'Mutation Description AA',
       'ONTOLOGY_MUTATION_CODE', 'GENOMIC_MUTATION_ID',
       'Mutation genome position GRCh37', 'Mutation genome position GRCh38',
       'COSMIC_SAMPLE_TESTED', 'COSMIC_SAMPLE_MUTATED', 'DISEASE',
       'WGS_DISEASE', 'EXAC_AF', 'EXAC_AFR_AF', 'EXAC_AMR_AF', 'EXAC_ADJ_AF',
       'EXAC_EAS_AF', 'EXAC_FIN_AF', 'EXAC_NFE_AF', 'EXAC_SAS_AF',
       'GNOMAD_EXOMES_AF', 'GNOMAD_EXOMES_AFR_AF', 'GNOMAD_EXOMES_AMR_AF',
       'GNOMAD_EXOMES_ASJ_AF', 'GNOMAD_EXOMES_EAS_AF', 'GNOMAD_EXOMES_FIN_AF',
       'GNOMAD_EXOMES_NFE_AF', 'GNOMAD_EXOMES_SAS_AF', 'GNOMAD_GENOMES_AF',
       'GNOMAD_GENOMES_AFR_AF', 'GNOMAD_GENOMES_AMI_AF',
       'GNOM

In [21]:
# both use MT for mitochondria (not M)
hgnc_bed_map = pickleLoad('../reference_genome/hgnc_bed_map.pkl')
hgnc_bed_map['GRCh37']


loading data from ../reference_genome/hgnc_bed_map.pkl


{'HGNC:37102': ('chr1', 11874, 14409),
 'HGNC:38034': ('chr1', 14362, 29370),
 'HGNC:50039': ('chr1', 17369, 17436),
 'HGNC:35294': ('chr1', 30366, 30503),
 'HGNC:32334': ('chr1', 34611, 36081),
 'HGNC:14822': ('chr1', 52453, 53396),
 'HGNC:31276': ('chr1', 63016, 63885),
 'HGNC:14825': ('chr1', 65419, 71585),
 'HGNC:51705': ('chr1', 126642, 129225),
 'HGNC:48835': ('chr1', 131068, 134836),
 'HGNC:48063': ('chr1', 157784, 157887),
 'HGNC:51722': ('chr1', 164689, 164792),
 'HGNC:35827': ('chr1', 228262, 228787),
 'HGNC:37756': ('chr1', 328518, 332282),
 'HGNC:51700': ('chr1', 334127, 334504),
 'HGNC:31275': ('chr1', 367659, 368597),
 'HGNC:43955': ('chr1', 379067, 379573),
 'HGNC:36176': ('chr1', 470896, 471368),
 'HGNC:42092': ('chr1', 564442, 564820),
 'HGNC:42129': ('chr1', 565020, 566066),
 'HGNC:52014': ('chr1', 566454, 567996),
 'HGNC:54043': ('chr1', 567995, 568065),
 'HGNC:52028': ('chr1', 568137, 568818),
 'HGNC:44571': ('chr1', 568915, 569121),
 'HGNC:44575': ('chr1', 569076, 

In [28]:
negatives['Mutation genome position GRCh37']

1            1:51121147-51121147
4            1:51061836-51061836
5            1:51032798-51032798
6            1:50956313-50956313
7            1:50956317-50956317
                   ...          
5419322     21:37270507-37270507
5419324      X:56101173-56101173
5419325     15:83207668-83207668
5419326     15:83207649-83207649
5419327    1:154585073-154585073
Name: Mutation genome position GRCh37, Length: 3722140, dtype: object